In [1]:
import torch as t
from mcs import MCS
from utils import indep_chkboard_sets
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [8]:
t.autograd.set_detect_anomaly(False)
test = t.zeros(1,32,32)
test[0,16,16] += 1

checkerboard_sets = indep_chkboard_sets(test, tile_height=4, tile_width=4)
target_vol = 1.
temperature = t.tensor(1.)
temperature.requires_grad_()

states = [test.detach().clone().squeeze().numpy()]
for i in tqdm(range(1000)):
    test, steps = MCS(test, checkerboard_sets, target_vol, temperature)
    states += steps
    if t.sum(test) == 0 or t.sum(test) > 2:
        print("ISSUE DETECTED, STOP SIM",t.sum(test), test)
        break

imgs = [Image.fromarray((1-state)*255) for state in states]
imgs[0].save("t_1_reflective.gif", save_all=True, append_images=imgs[1:], duration=10, loop=100)

100%|██████████| 1000/1000 [00:17<00:00, 56.05it/s]
